In [13]:
import os
# 파이썬에서 파일 시스템 및 운영 체제와 상호 작용하는데 필요한 기능을 제공
import time
import pandas as pd
import pymysql

In [14]:
# current_directory = os.getcwd()
# 대화형 셸(like 주피터노트북)에서는 __file__이 정의되지 않아 위에 방법으로 해야함

In [15]:
class LOAD_CSV():
    def __init__(self):
#   __init__ 메서드: 클래스 생성자 역할. 그래서 여기서는 클래스 초기화 작업을 수행했다고 보면 됨.
# 아무런 인자를 받고있지 않으나, 'self'라는 객체 자신을 받고 있다.
        self.main()
      # 클래스 내에서 main()이라는 메서드를 호출
      # self라는 객체의 main 메서드 호출
    def main(self):
#     "main은 코드실행의 시작점을 알려주는 메서드로 보입니다."
        start = time.time()
        print('start')
        
# 클래스는 인스턴스(객체)를 생성(인스턴스화)하여 그 아래에서 메서드를 호출.
# 객체 생성 후에는 해당 클래스의 메서드를 호출하여 객체가 수행할 작업을 지정할 수 있음.

        # CSV 파일이 들어있는 디렉토리 경로
        directory = './data/국토교통부_전국아파트매매'

        # pymysql을 사용하여 MariaDB에 연결
        self.conn = pymysql.connect(host='127.0.0.1', user='user', password='user1234', db='opencc_1', charset='utf8')
        # 커서 생성
        self.cur = self.conn.cursor()
        print('conn, cursor complete')
        
    # --디렉토리 내 모든 CSV 파일 가져오기--
        for foldername in os.listdir(directory):
        # 주어진 디렉토리 'directory'의 모든 항목을 반복. 'os.listdir()' 함수는 주어진 디렉토리에 포함된 모든 파일과 폴더 이름을 리스트로 반환.
            print(foldername)
            folder_path = os.path.join(directory, foldername)
            # 현재 폴더의 경로 생성. 'os.path.join()' 함수를 사용하여 'directory'와 'foldername' 연결, 전체 경로 만듦. 
            # 'directory'는 폴더가 있는 주소(예: '/home/user'). 그래서 os.path.join()으로 directory(경로) + foldername(경로) 해주면 폴더의 전체 경로가 됨.
            # 위에서는 그걸 folder_path에 저장.
            if os.path.isdir(folder_path): 
            # 현재 돌고있는 폴더 전체경로가 디렉토리인지 확인
                for filename in os.listdir(folder_path):
                # 현재 돌고 있는 폴더 내의 모든 파일 반복
                    print(foldername, filename)
                    if filename.endswith(".csv"):
                        r_filename = os.path.splitext(filename)[0]
                        # os.path.splitext(): 파일이름과 확장자를 나누어주는 함수. [0] 부분에는 파일이름이 있다.
                        filepath = os.path.join(folder_path, filename)
                        print(f"Reading file: {filename}")
                        
        # 위 코드는 주어진 디렉토리 내의 모든 폴더를 반복하고 각 폴더 내의 모든 파일을 검색하여 ".csv" 확장자를 가진 파일을 찾고, 해당 파일들을 읽는 작업을 수행.
        
                        # CSV 파일을 데이터프레임으로 읽기
                        df = pd.read_csv(filepath, encoding='utf8', skiprows=15)
                        # 주어진 경로에서 csv 파일을 읽어 데이터프레임으로 로드. 인코딩 방식=cp949. 처음 15줄 건너뛰고 읽기.
                        # 테이블이 16째줄부터 시작
                        df['거래금액(만원)'] = df['거래금액(만원)'].str.replace(',','').apply(int)
                        # '거래금액(만원)' 열의 각 셀에서 쉼표를 공백으로 바꾸고 (예: 1,000 -> 1 000) 그 다음 int를 apply해서 정수형으로 변환.
                        # str.remove는 없어서 바로 쉼표만 없앨 수 없고, 공백으로 바꿨다가 정수형 변환 처리 해야되나봄.
                        print(f'df read int {filename} complete')
                        # csv 파일 읽고 정수형으로 변환하는 작업이 완료되었음을 출력.
                        self.db_create(r_filename)
                        # .db_create(메서드)를 통해 데이터베이스에 새로운 테이블 만듦.
                        # 테이블 이름 - csv 파일에서 확장자 제외한 부분인 r_filename
                        print(f'db_create {r_filename} complete')
                        # 데이터베이스에 테이블 생성하는 작업 완료되었음을 출력.

                        self.db_insert(r_filename, df)
                        #db_insert: 데이터베이스에 데이터프레임 삽입하는 메서드.
                        # 테이블 이름 - r_filename / 데이터는 df에 저장되어 있음.
                        # 아마 위에 만든 테이블에 df를 삽입하는 거겠지?
                        print(f'db_insert {r_filename} complete')
                        # 데이터베이스에 데이터 삽입하는 작업이 완료되었음을 출력.
                        
        # 변경사항 커밋
        self.conn.commit()

        # 연결 종료
        self.conn.close()

        end = time.time()
        print('end')
        total = end - start
        print(f'{int(total)//60:02}:{int(total)%6:02}{str(total-int(total))[1:]}')
        # total은 초로 저장되어 있음. 그래서 첫번째는 분을 구하는 식이고, ':02'를 통해 두 자리수로 표현되게 만들어줌.
        # if total == 123.456, total-int(total) == .456 / 거기서 첫번째 문자 이후만 가져오는 거니까 456이 선택됨.

        
        def db_create(self, table):
            self.cur.execute(f"""
            CREATE TABLE IF NOT EXISTS {table} (
                transaction_id INT AUTO_INCREMENT PRIMARY KEY,
                시군구 VARCHAR(255),
                번지 VARCHAR(255),
                본번 INT,
                부번 INT,
                단지명 VARCHAR(255),
                전용면적 FLOAT,
                계약년월 INT,
                계약일 INT,
                거래금액 INT,
                동 VARCHAR(255),
                층 INT,
                매수자 VARCHAR(255),
                매도자 VARCHAR(255),
                건축년도 INT,
                도로명 VARCHAR(255),
                해제사유발생일 VARCHAR(255),
                거래유형 VARCHAR(255),
                중개사소재지 VARCHAR(255),
                등기일자 VARCHAR(255)
            );
            """)
        # 'self.cur.execute()' 라는 메서드 사용하여 SQL문 실행.
        # table 이름 == table
        # AUTO_INCREMENT 키워드를 사용하여 transaction_id 컬럼을 자동으로 증가되는 정수(primary key)로 지정.
        
        # 'VARCHAR(255)'는 최대 255자의 문자열을 저장할 수 있는 VARCHAR 형식. 그런데 실제로 입력된 문자열의 길이가 이것보다 짧다면, 실제로 할당된 공간은 문자열의 길이만큼임.
        

    def db_insert(self, table, df):
        # df의 각 행을 'table'에 삽입하는 함수 
        for index, row in df.iterrows():
        # 데이터프레임의 각 행을 반복하며, 각 행을 데이터베이스에 삽입.
            self.cur.execute(f"""
                INSERT INTO {table} (""" + """
                    시군구,
                    번지,
                    본번,
                    부번,
                    단지명,
                    전용면적,
                    계약년월,
                    계약일,
                    거래금액,
                    동,
                    층,
                    매수자,
                    매도자,
                    건축년도,
                    도로명,
                    해제사유발생일,
                    거래유형,
                    중개사소재지,
                    등기일자
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                row['시군구'],
                row['번지'],
                row['본번'],
                row['부번'],
                row['단지명'],
                row['전용면적(㎡)'],
                row['계약년월'],
                row['계약일'],
                row['거래금액(만원)'],
                row['동'],
                row['층'],
                row['매수자'],
                row['매도자'],
                row['건축년도'],
                row['도로명'],
                row['해제사유발생일'],
                row['거래유형'],
                row['중개사소재지'],
                row['등기일자']
            ))
            
            # VALUES 절에 테이블의 각 열 값을 %s(위치지정자)로 지정한다고 명시되어있음.
            # %s 위치 지정자는 SQL Injection 공격을 방지하기 위해 사용됨. %s는 실제 값으로 바인딩되어 SQL 쿼리가 실행될 때 적절한 값으로 대체됨.
            # VALUES 다음 튜플이 나오는 것 -> 이 튜플은 SQL 쿼리에서 사용되는 값들을 나타내며, 데이터프레임의 각 행(row)에서 추출된 열(column)의 값들을 포함.
            #  row['시군구'], row['번지'], row['본번'], 등과 같은 각각의 값은 데이터프레임의 특정 행(row)에서 가져온 열(column)의 값을 나타냄(row 이름이 아니다!). 이 값들이 SQL 쿼리의 %s 위치 지정자에 바인딩되어 실행될 때 적절한 위치에 삽입됨.
            
if __name__ == '__main__':
    LC = LOAD_CSV()
# 만약 현재 스크립트가 직접 실행되었을 경우에만 (if __name__ == '__main__':) 아래 코드블록을 실행.
# 그러면 LOAD_CSV()의 객체를 생성해서 스크립트 실행.
# 모듈로 임포트되어 사용될 경우에는 실행되지 않음으로써, 코드 중복 실행과 그에 따른 성능 문제 가능성을 없앨 수 있다.


# 모듈은 코드의 조각을 재사용하기 위해 만들어진 파일이고, 스크립트는 특정 작업을 수행하기 위해 실행되는 파일.


start


OperationalError: (1049, "Unknown database 'opencc_1'")